## LinkedIn Profile Scraper in Python using LLM

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [4]:
driver = webdriver.Chrome()

In [5]:
driver.get('https://www.linkedin.com/login')

In [6]:
driver.title

'LinkedIn Login, Sign in | LinkedIn'

In [7]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.getenv('EMAIL'))

In [8]:
password = driver.find_element(By.ID, 'password')
password.send_keys(os.getenv('PASSWORD'))

In [9]:
password.submit()

In [10]:
## MAKE SURE TO USE ONLY THIS URL TO AVOID BEING STUCK IN ERRORS

url = "https://www.linkedin.com/in/laxmimerit"

driver.get(url)

In [11]:
page_source = driver.page_source

In [12]:
soup = BeautifulSoup(page_source, 'lxml')

In [17]:
# soup.get_text()

In [33]:
profile = soup.find('main', {'class': 'KtcnatnKNtVpJiMGcZXtzztRsVfgOORs'})


In [35]:
# print(profile.get_text())

In [36]:
sections = profile.find_all('section', {'class': 'artdeco-card'})

In [37]:
len(sections)

17

In [26]:
# sections

In [39]:
sections_text = [section.get_text() for section in sections]
sections_text

['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n              Click to upgrade to Premium\n            \n\n\n        Laxmi Kant has a {:badgeType} account\n      \n\n\n\n\n\n \n\n\n\n\n    test\n\n\n \n\n\n\n\n\n\n\n\nLaxmi Kant Tiwari\n\n\n\n\n \n\n\n\n\n\n          1st degree connection\n      \n\n        1st\n      \n\n\n \n          Gen AI in Finance & Investment Services | Data Scientist | IIT Kharagpur | Asset Management | AI-Driven Financial Modeling | Search Ranking | NLP Python BERT AWS Elasticsearch GNN SQL LLM | AI in Investment Strategies\n        \n \n\n \n\n\n\n\n\n        Indian Institute of Technology, Kharagpur\n      \n \n\n\n\n\n\n \n          Mumbai, Maharashtra, India\n        \n\n\n              Contact info\n            \n\n\n\n\n\n\n26,901 followers\n        \n\n\n500+ connections\n            \n\n\n\n\n\n\n\n\n \n\n\n\n\n\n \n\n\n\n \n\nRamnivas Verma, Aayush Saxena, and 5 other mutual connectionsRamnivas Verma, Aayush Saxena, and 5 other

In [40]:
import re

# remove multiple new lines and tabs
def clean_text(text):
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\t+', '\t', text)
    text = re.sub(r'\t\s+', ' ', text)
    text = re.sub(r'\n\s+', '\n', text)
    return text

In [42]:
sections_text = [clean_text(section) for section in sections_text]
sections_text

['\nClick to upgrade to Premium\nLaxmi Kant has a {:badgeType} account\ntest\nLaxmi Kant Tiwari\n1st degree connection\n1st\nGen AI in Finance & Investment Services | Data Scientist | IIT Kharagpur | Asset Management | AI-Driven Financial Modeling | Search Ranking | NLP Python BERT AWS Elasticsearch GNN SQL LLM | AI in Investment Strategies\nIndian Institute of Technology, Kharagpur\nMumbai, Maharashtra, India\nContact info\n26,901 followers\n500+ connections\nRamnivas Verma, Aayush Saxena, and 5 other mutual connectionsRamnivas Verma, Aayush Saxena, and 5 other mutual connections\nMessage\nMore\nSend profile in a message\nSave to PDF\nRequest a recommendation\nRecommend\nUnfollow\nRemove Connection\nReport / Block\nAbout this profile\nAbout this profile\n',
 "\nHighlightsHighlights\nLaxmi Kant is a new connectionLaxmi Kant is a new connection\n“Hi Laxmi Kant, it's great to connect with you. Looking forward to staying in touch.”“Hi Laxmi Kant, it's great to connect with you. Looking fo

In [43]:
def remove_duplicates(text):
    lines = text.split('\n')
    new_lines = []
    for line in lines:
        if line[:len(line)//2] == line[len(line)//2:]:
            new_lines.append(line[:len(line)//2])
        else:
            new_lines.append(line)

    return '\n'.join(new_lines)

In [44]:
sections_text = [remove_duplicates(section) for section in sections_text]

In [45]:
print(sections_text[1])


Highlights
Laxmi Kant is a new connection
“Hi Laxmi Kant, it's great to connect with you. Looking forward to staying in touch.”
Say hello



In [46]:
sections_text

['\nClick to upgrade to Premium\nLaxmi Kant has a {:badgeType} account\ntest\nLaxmi Kant Tiwari\n1st degree connection\n1st\nGen AI in Finance & Investment Services | Data Scientist | IIT Kharagpur | Asset Management | AI-Driven Financial Modeling | Search Ranking | NLP Python BERT AWS Elasticsearch GNN SQL LLM | AI in Investment Strategies\nIndian Institute of Technology, Kharagpur\nMumbai, Maharashtra, India\nContact info\n26,901 followers\n500+ connections\nRamnivas Verma, Aayush Saxena, and 5 other mutual connections\nMessage\nMore\nSend profile in a message\nSave to PDF\nRequest a recommendation\nRecommend\nUnfollow\nRemove Connection\nReport / Block\nAbout this profile\nAbout this profile\n',
 "\nHighlights\nLaxmi Kant is a new connection\n“Hi Laxmi Kant, it's great to connect with you. Looking forward to staying in touch.”\nSay hello\n",
 '\nAbout\nDemonstrated 8+ years of expertise in Advanced Analytics and Machine Learning, currently leading strategic AI initiatives at Linedat

### Profile Data Parsing using LLM

In [60]:
from langchain_ollama import ChatOllama

from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)



from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
# model = 'llama3.2:3b'
model = "qwen2.5:7b"

llm = ChatOllama(base_url=base_url, model=model)


system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who answer LinkedIn profile parsing related 
                                                    user question based on the provided profile text data.""")

def ask_llm(prompt):
    prompt = HumanMessagePromptTemplate.from_template(prompt)

    messages = [system, prompt]
    template = ChatPromptTemplate(messages)

    qna_chain = template | llm | StrOutputParser()

    return qna_chain.invoke({})

In [61]:
llm

ChatOllama(model='qwen2.5:7b', base_url='http://localhost:11434')

In [62]:
ask_llm("hello")

'Hello! How can I assist you with parsing or analyzing a LinkedIn profile? Please provide the relevant profile text data so I can help you specifically.'

In [47]:
template = """
Extract and return the requested information from the LinkedIn profile data in a concise, point-by-point format (up to 5 points). Avoid preambles or any additional context.

### LinkedIn Profile Data:
{}

### Information to Extract:
Extract '{}' in bullet points, limiting the output to 5 points. Provide only the necessary details.
Remember, It is LinkedIn profile data.

### Extracted Data:"""

context = sections_text[0]
k = "Name and Headline"

prompt = template.format(context, k)
response = ask_llm(prompt)

In [48]:
print(response)

• Laxmi Kant 
• Gen AI in Finance & Investment Services
• IIT Kharagpur
• Mumbai, Maharashtra, India
• 26,723 followers


In [59]:
section_keys = ['Name and Headline']
for section in sections_text[1:]:
    # print(section.strip().split('\n'))
    section_keys.append(section.strip().split('\n')[0])

section_keys

['Name and Headline',
 'Highlights',
 'About',
 'Featured',
 'Activity',
 'Experience',
 'Education',
 'Licenses & certifications',
 'Projects',
 'Skills',
 'Recommendations',
 'Patents',
 'Courses',
 'Honors & awards',
 'Languages',
 'Interests',
 'Causes']

In [63]:
# sections_text
responses = {}

for k,context in zip(section_keys, sections_text):
    prompt = template.format(context, k)
    response = ask_llm(prompt=prompt)
    responses[k] = response

In [64]:
print(responses)

{'Name and Headline': '- Name: Laxmi Kant\n- Headline: 1st degree connection | Gen AI in Finance & Investment Services | Data Scientist | IIT Kharagpur | Asset Management | AI-Driven Financial Modeling | Search Ranking | NLP Python BERT AWS Elasticsearch GNN SQL LLM | AI in Investment Strategies', 'Highlights': '- Laxmi Kant has viewed your profile\n- Curious about who else viewed your profile? Unlock for more opportunities.\n- See all profile views', 'About': '- Demonstrated 8+ years of expertise in advanced analytics as an AVP in Data Science.\n- Strategically led impactful initiatives in Customer Behavior Modeling, resulting in a substantial 30% increase in customer retention and a 10% reduction in advertising spending.\n- Pioneered the development and implementation of a Machine Learning product relevance ranking system, contributing to a remarkable 30% increase in conversion rates and a 20% growth in revenue.\n- Applied innovative techniques, including Social Graph Analytics and P

In [65]:
import json

with open('linkedin_profile_data.json', 'w') as f:
    json.dump(responses, f, indent=4)

### Second Level of LLM Call

In [66]:
template = """You are provided with LinkedIn profile data in JSON format.
            Parse the data according to the specified schema, correct any spelling errors,
            and condense the information if possible.

### LinkedIn Profile JSON Data:
{context}

### Schema You need to follow:
You need to extract
Name:
Headline:
About:
Experience:
Education:
Skills:
Projects:
Summary:

Do not return preambles or any other information.
### Parsed Data:"""

prompt = template.format(context=responses).replace("{", "{{").replace("}", "}}")
response = ask_llm(prompt=prompt)
# response

In [67]:
print(response)

Name: Laxmi Kant

Headline: 1st degree connection | Gen AI in Finance & Investment Services | Data Scientist | IIT Kharagpur | Asset Management | AI-Driven Financial Modeling | Search Ranking | NLP Python BERT AWS Elasticsearch GNN SQL LLM | AI in Investment Strategies

About: Demonstrated 8+ years of expertise in advanced analytics as an AVP in Data Science. Strategically led impactful initiatives in Customer Behavior Modeling, resulting in a substantial 30% increase in customer retention and a 10% reduction in advertising spending. Pioneered the development and implementation of a Machine Learning product relevance ranking system, contributing to a remarkable 30% increase in conversion rates and a 20% growth in revenue. Applied innovative techniques, including Social Graph Analytics and Predictive Modeling, achieving a 35% improvement in recommendation conversion rates and a 40% increase in customer care calling conversion rates. Revolutionized customer segmentation and ranking, real